# Olist Database Creation

Цель этапа - загрузить данные в базу данных для их дальнейшей обработки.

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

In [2]:
db_path = "olist_database.db"
conn = sqlite3.connect(db_path)

In [3]:
csv_dir = Path("../data")

for csv_file in csv_dir.glob("*.csv"):
    table_name = csv_file.stem  # имя файла без .csv
    
    df = pd.read_csv(csv_file)
    
    df.to_sql(
        table_name,
        conn,
        if_exists="replace",
        index=False
    )
    
    print(f"Загружена таблица: {table_name}, строк: {len(df)}")

Загружена таблица: olist_customers_dataset, строк: 99441
Загружена таблица: olist_geolocation_dataset, строк: 1000163
Загружена таблица: olist_orders_dataset, строк: 99441
Загружена таблица: olist_order_items_dataset, строк: 112650
Загружена таблица: olist_order_payments_dataset, строк: 103886
Загружена таблица: olist_order_reviews_dataset, строк: 99224
Загружена таблица: olist_products_dataset, строк: 32951
Загружена таблица: olist_sellers_dataset, строк: 3095
Загружена таблица: product_category_name_translation, строк: 71


**Сравним количество строк загруженных и количество строк в датасетах, чтобы убедиться, что данные загружены в базу в полном объеме:**

In [8]:
csv_dir = Path("../data")

rows_info = []

for csv_file in csv_dir.glob("*.csv"):
    df = pd.read_csv(csv_file)
    
    rows_info.append({
        "table": csv_file.stem,
        "rows": df.shape[0]
    })

result_df = pd.DataFrame(rows_info).sort_values("table")
result_df

,table,rows
0,olist_customers_dataset,99441
1,olist_geolocation_dataset,1000163
3,olist_order_items_dataset,112650
4,olist_order_payments_dataset,103886
5,olist_order_reviews_dataset,99224
2,olist_orders_dataset,99441
6,olist_products_dataset,32951
7,olist_sellers_dataset,3095
8,product_category_name_translation,71


**Загрузилось все верно, теперь посмотрим на данные:**

In [6]:
pd.read_sql(
    "SELECT * FROM olist_orders_dataset LIMIT 5",
    conn
)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [9]:
pd.read_sql(
    "SELECT * FROM olist_products_dataset LIMIT 5",
    conn
)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


**В двух рандомно выбранных датасетах данные отражаются. Проверим уникальность ключей:**

In [11]:
pd.read_sql(
    "SELECT COUNT(DISTINCT order_id) FROM olist_orders_dataset",
    conn
)

,COUNT(DISTINCT order_id)
0,99441


In [13]:
pd.read_sql(
    "SELECT COUNT(*) FROM olist_orders_dataset",
    conn
)

,COUNT(*)
0,99441


Отметим, что в таблице заказов 99441 уникальный заказ, что равняется кол-ву записей в таблице

In [12]:
pd.read_sql(
    "SELECT COUNT(DISTINCT order_id) FROM olist_order_reviews_dataset",
    conn
)

,COUNT(DISTINCT order_id)
0,98673


In [19]:
pd.read_sql(
    "SELECT COUNT(*) FROM olist_order_reviews_dataset",
    conn
)

,COUNT(*)
0,99224


In [18]:
pd.read_sql(
    "SELECT order_id, COUNT(*) AS reviews_cnt FROM olist_order_reviews_dataset GROUP BY order_id HAVING COUNT(*) > 1",
    conn
)

,order_id,reviews_cnt
0,0035246a40f520710769010f752e7507,2
1,013056cfe49763c6f66bda03396c5ee3,2
2,0176a6846bcb3b0d3aa3116a9a768597,2
3,02355020fd0a40a0d56df9f6ff060413,2
4,029863af4b968de1e5d6a82782e662f5,2
...,...,...
542,fd95ae805c63c534f1a64589e102225e,2
543,fe041ba1c9f54016432fa6ee91709dbc,2
544,ff763b73e473d03c321bcd5a053316e8,2
545,ff850ba359507b996e8b2fbb26df8d03,2


In [22]:
pd.read_sql(
    "SELECT * FROM olist_order_reviews_dataset WHERE order_id = '0035246a40f520710769010f752e7507'",
    conn
)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,2a74b0559eb58fc1ff842ecc999594cb,0035246a40f520710769010f752e7507,5,None,Estou acostumada a comprar produtos pelo barat...,2017-08-25 00:00:00,2017-08-29 21:45:57
1,89a02c45c340aeeb1354a24e7d4b2c1e,0035246a40f520710769010f752e7507,5,None,None,2017-08-29 00:00:00,2017-08-30 01:59:12


Отметим, что в таблице с отзывами некоторые заказы имеют по несколько отзывов. Нужно будет обязательно это учитывать при последующих джоинах, так как будут всплывать дубликаты.

In [23]:
pd.read_sql(
    "SELECT COUNT(DISTINCT order_id) FROM olist_order_items_dataset",
    conn
)

,COUNT(DISTINCT order_id)
0,98666


In [24]:
pd.read_sql(
    "SELECT COUNT(*) FROM olist_order_items_dataset",
    conn
)

,COUNT(*)
0,112650


Отметим, что в одном заказе может быть несколько товаров, поэтмоу логично, что общее число строк > уникальных заказов.

In [25]:
pd.read_sql(
    "SELECT COUNT(DISTINCT order_id) FROM olist_order_payments_dataset",
    conn
)

,COUNT(DISTINCT order_id)
0,99440


In [26]:
pd.read_sql(
    "SELECT COUNT(*) FROM olist_order_payments_dataset",
    conn
)

,COUNT(*)
0,103886


Здесь также отметим, что один заказ может оплачиваться частями, поэтому общее кол-во строк больше.

Далее, перейдем к проверке джоинов. Но для начала зафиксируем связи между таблицами, чтобы не запутаться.

## Связи между таблицами

- orders → customers  
  many-to-one (один клиент может сделать несколько заказов)

- orders → order_items  
  one-to-many (один заказ может содержать несколько товаров)

- orders → order_payments  
  one-to-many (один заказ может иметь несколько платежей)

- orders → order_reviews  
  one-to-many (одному заказу может соответствовать несколько отзывов)

- order_items → products  
  many-to-one (один товар может входить в разные заказы)

- order_items → sellers  
  many-to-one (один продавец может продавать разные товары)


In [28]:
pd.read_sql(
    "SELECT o.order_id, c.customer_id, o.order_status  FROM olist_orders_dataset o JOIN olist_customers_dataset c ON o.customer_id = c.customer_id"
    ,
    conn
)

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered


При соединении CUSTOMERS и ORDERS, как и ожидалось, количество строк не увеличилось.

In [30]:
pd.read_sql(
    "SELECT o.order_id, oi.product_id, oi.price FROM olist_orders_dataset o JOIN olist_order_items_dataset oi ON o.order_id = oi.order_id"
    ,
    conn
)

,order_id,product_id,price
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,aa4383b373c6aca5d8797843e5594415,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,d0b61bfb1de832b15ba9d266ca96e5b0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,65266b2da20d04dbe00c5c2d3bb7859e,19.90
...,...,...,...
112645,63943bddc261676b46f01ca7ac2f7bd8,f1d4ce8c6dd66c47bbaa8c6781c2a923,174.90
112646,83c1379a015df1e13d02aae0204711ab,b80910977a37536adeddd63663f916ad,205.99
112647,11c177c8e97725db2631073c19f07b62,d1c427060a0f73f6b889a5c7c61f2ac4,179.99
112648,11c177c8e97725db2631073c19f07b62,d1c427060a0f73f6b889a5c7c61f2ac4,179.99


Количество строк = количеству строк в order_items

In [35]:
pd.read_sql(
    'SELECT o.order_id, ore.review_score FROM olist_orders_dataset o JOIN olist_order_reviews_dataset ore ON o.order_id = ore.order_id', conn
    
)

,order_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,4
1,53cdb2fc8bc7dce0b6741e2150273451,4
2,47770eb9100c2d0c44946d9cf07ec65d,5
3,949d5b44dbf5de918fe9c16f97b45f8a,5
4,ad21c59c0840e6cb83a9ceb5573f8159,5
...,...,...
99219,9c5dedf39a927c1b2549525ed64a053c,5
99220,63943bddc261676b46f01ca7ac2f7bd8,4
99221,83c1379a015df1e13d02aae0204711ab,5
99222,11c177c8e97725db2631073c19f07b62,2


Количество строк увеличилось, так как один заказ может иметь несколько отзывов.

## Итоги 2 этапа:
- Данные загружены в базу данных
- Связи между таблицами проверены
- JOIN-ы работают корректно
- База готова для следующего этапа - feature engineering